In [ ]:
import warnings

warnings.filterwarnings("ignore")

# EDA of "King County Housing Data"

In this Notebook I will perform an exploratory analysis of the "King County Housing Data". 
The data was exported to a `.csv` file from an PostgreSQL database using `psycopg2`.

## Data cleaning
Importing the necessary libraries:

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno 

Creating the Dataframe and looking at the types of data and if everything is imported correctly:

In [ ]:
df = pd.read_csv('eda.csv')
df.info()
df.head()

Converting `floats` to `ints` where applicable and converting the date to datetime:

In [ ]:
df['yr_renovated'] = df['yr_renovated'].div(10) # year_renovated was wrongly formatted (i.e. 1991 = 19910)

# converts the floats to ints where applicable
df = df.convert_dtypes() 

# converting date to datetime
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

Filling `NaN` values with `0` after checking the columns with `msno` and looking at the values in each column:

In [ ]:
print(df['waterfront'].unique())
print(df['view'].unique())
print(str(df['sqft_basement'].min()) + " - "  + str(df['sqft_basement'].max()))
print(str(df['yr_renovated'].min()) + " - "  + str(df['yr_renovated'].max()))
msno.matrix(df)

In [ ]:
# filling the NaN values with 0
df.fillna(0,inplace=True)


Finally renaming 'date' to remove ambiguity:

In [ ]:
df.rename(columns={'date':'sell_date'},inplace=True)

## Data Analysis

Hypotheses about the data:
 1. The condition and grade of a house have a correlation with the price
 2. Expensive houses are all close to each other
 3. The price for a house with a good view or near the waterfront are higher than the median

I will also make three recommendations for my client, Timothy Stevens. He is looking to sell houses at the best time of the year and is open to renovations if they improve the selling price.


### Exploring the data
Importing additional libararies:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from urllib.request import urlopen
import json


FIrst let's look at the data to get an inital picture:

In [ ]:
# looking at the metrics of the table:
df.describe()


**Initial thoughts:** 
* Most houses aren't at the water, don't have an extraordinary view and haven't been renovated
* They have around 3 bedrooms and 2 bathrooms and a living area of 2080 sqft
* The mean age of the houses is 52 years
* The mean selling price is 540 000 $

To reduce scope of the exploration we will create a new dataframe:

In [ ]:
df_expl = df.drop(columns=['lat','long','sqft_lot15','sqft_living15',
                           'bedrooms','bathrooms','floors','sell_date','id','waterfront','view','sqft_basement','sqft_above'])


Generating a correlation matrix and heatmap:

In [ ]:
# create a correlation matrix
corr_mtrx1 = df_expl.corr().round(2)

plt.style.use('fast')
# plot heatmap using seaborn
ax = sns.heatmap(corr_mtrx1, linewidths = .5, annot = True,vmin=-1, cmap= 'coolwarm' )
plt.show()

Looking at the relationship between price, grade and condition:

In [ ]:
# creating a scatterplot with seaborn
sctr_plt1 = sns.scatterplot(x=df_expl['condition'],y=df_expl['price'])
sns.despine(right=True,top=True)
sctr_plt1.set_yticklabels('{:,.2f}'.format(x) + 'M' for x in sctr_plt.get_yticks()/1000000)
sctr_plt1.set_title('relationship between condition and price'.title())
plt.show()

In [ ]:
# creating a scatterplot with seaborn
sctr_plt2 = sns.scatterplot(x=df_expl['grade'],y=df_expl['price'])
sns.despine(right=True,top=True)
sctr_plt2.set_yticklabels('{:,.2f}'.format(x) + 'M' for x in sctr_plt.get_yticks()/1000000)
sctr_plt2.set_title('relationship between grade and price'.title())
plt.show()

We can see that there is little correlation between price and condition, but price and grade are connected.

Since only 3% of the houses have been renovated, let's see if the correlation changes when we look only at renovated properties:

In [ ]:
# creating a new dataframe containing price, grade, condition and renovated properties
df_reno = df_expl[['price','grade','condition','yr_renovated','sqft_living']].astype(float).query('yr_renovated != 0')




Looking at the histogram to see when properties have been renovated:

In [ ]:
histgrm = sns.histplot(data=df_reno,x=df_reno['yr_renovated'],bins=69)
sns.despine(right=True,top=True)
histgrm.set(xlabel='Year renovated')

df_reno['yr_renovated'].describe()

We can see, that 75% of the houses have been renovated after 1987, therefore we will now look at these to see if there is any correlation to the grade/ condition or price: 

In [ ]:
# create a correlation matrix
corr_mtrx2 = df_reno.query('yr_renovated >= 1987').corr().round(2)

# plot heatmap using seaborn
ax = sns.heatmap(corr_mtrx2, linewidths = .5, annot = True,vmin=-1, cmap= 'coolwarm' )
plt.show()

We can see that there is little correlation between price, condition and when the property was renovated. Also there's no correlation between condition and grade.
A scatterplot shows the correlation between price & grade and price & condition.


In [ ]:
df_reno = df_reno.query('yr_renovated >= 1987')
# creating a scatterplot with seaborn
sctr_plt3 = sns.scatterplot(x=df_reno['condition'],y=df_reno['price'])
sns.despine(right=True,top=True)
sctr_plt3.set_yticklabels('{:,.2f}'.format(x) + 'M' for x in sctr_plt.get_yticks()/1000000)
sctr_plt3.set_title('Relationship between condition and price \n for houses renovated after 1987'.title())
plt.show()

In [ ]:
# creating a scatterplot with seaborn
sctr_plt4 = sns.scatterplot(x=df['grade'],y=df['price'])
sns.despine(right=True,top=True)
sctr_plt4.set_yticklabels('{:,.2f}'.format(x) + 'M' for x in sctr_plt.get_yticks()/1000000)
sctr_plt4.set_title('Relationship between price and grade \n for houses renovated after 1987'.title())
plt.show()


**Conclusion for hypothesis 1:**
The data does not show a correlation between grade & conditon and condition & price, but there is correlation between price & grade.

Next we'll look at the relation between price and location:
First we'll plot the location of all the properties with coloration according to price:

In [ ]:
# plot scatterplot using seaborn
sns.scatterplot(x=df['long'], y=df['lat'], hue=df['price'].astype(float),s=6,palette='viridis')
sns.despine(right=True,top=True)

We can see, that the more expensive properties are close to each other and concentrated around the Bellevue area.

In [ ]:
# create a new index for the sorted plot
median_prices = df.groupby('zipcode')['price'].median().sort_values(ascending=False)
median_prices_index = median_prices.index

#plot sorted by median price of a zipcode
plt.figure(figsize=(12, 5))
bx_plt = sns.boxplot(data=df,x=df['zipcode'],y=df['price'],order=median_prices_index,palette='viridis_r')
sns.despine(right=True,top=True)

# changing the layout
bx_plt.set_xticklabels(bx_plt.get_xticklabels(),rotation = 45)
bx_plt.set_title('median price per zipcode'.title())

In [ ]:
print('90th percentile Median price over all zipcodes: ' + str(median_prices.quantile(q=0.90)) + ' $')
median_prices.head(8)

We can see that 7 zipcodes have a higher median price than 90% of all the zipcodes. But we can also see that not all of the most expensive houses are in the most expensive zipcode areas:


In [ ]:
# retrieving map data for the plot
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/wa_washington_zip_codes_geo.min.json') as response:
    geojson_data = json.load(response)

# Create a DataFrame to plot from the median prices
df_map = median_prices.to_frame()
df_map.reset_index(inplace=True)

In [ ]:

# Create a map figure using Plotly Express
fig = px.choropleth_mapbox(df_map, geojson=geojson_data, locations='zipcode', color='price',
                           color_continuous_scale='viridis',
                           mapbox_style='carto-positron',
                           featureidkey='properties.ZCTA5CE10',
                           zoom=7.8, center={'lat': 47.35, 'lon': -122.04},
                           opacity=0.7,hover_name='zipcode', hover_data={'zipcode':False}
                           )

# Update the layout
fig.update_layout(title_text='Data Overlay on King County Map',
                  margin={'r': 0, 't': 30, 'l': 0, 'b': 0})

# Display the figure
fig.show()

In [ ]:
# create data with the top 20 houses by price
top_20_prices = df.sort_values('price', ascending=False).head(20)
latitude_data = top_20_prices['lat'].tolist()
longitude_data = top_20_prices['long'].tolist()
latitude_data

hover_template = 'Latitude: %{lat}<br>Longitude: %{lon}<br>Price in $: %{text}'

# Create a scatter_mapbox trace for the layer with latitude-longitude markers
scatter_mapbox_trace = go.Scattermapbox(
    lat=latitude_data,  # List of latitude values
    lon=longitude_data,  # List of longitude values
    mode='markers',  # Set the marker mode
    marker=dict(size=6, color='white'),  
    hovertemplate=hover_template,
    text = top_20_prices['price'].astype(str))

scatter_mapbox_trace.showlegend = False
# Add the scatter_mapbox trace to the map figure
fig.add_trace(scatter_mapbox_trace)

# Update the layout
fig.update_layout(
    title='Data Overlay on King County Map with 20 most expensive houses',
    margin={'r': 0, 't': 30, 'l': 0, 'b': 0})

We can see that 16 of the 20 most expensive houses are in the 7 zipcode areas identified above.

**Conclusion for hypothesis 2:** The most expensive houses are concentrated in high median price areas.

On to the final hypothesis, examining the influence of the view and the location at the waterfront on the price:


In [ ]:
#creating a new dataframe with all houses that have a view better than 2 and are at the waterfront
df_nice_location = df.query('view > 2 and waterfront == 1')
median_prices_nice_loc = df_nice_location.groupby('zipcode')['price'].median().sort_values(ascending=False)
median_prices_nice_loc.name = 'price_nice_loc'

df_median_prices_loc = pd.merge(median_prices, median_prices_nice_loc, left_index=True, right_index=True, how='inner')
df_median_prices_loc.dropna(inplace=True)
df_median_prices_loc.reset_index(inplace=True)
df_median_prices_loc.eval('price_delta_loc = price_nice_loc - price',inplace=True)

In [ ]:
# creating barplot with seaborn
loc_plt = sns.barplot(data=df_median_prices_loc,x='zipcode',y='price_delta_loc',
                      order=df_median_prices_loc.sort_values('price_delta_loc',ascending=False).zipcode,
                      palette='viridis_r')
sns.despine(right=True,top=True)

#layout of the plot
loc_plt.set_title('median price difference between houses in good locations and all houses'.title())
loc_plt.set_ylabel('Price difference')
loc_plt.set_yticklabels('{:,.2f}'.format(x) + 'M' for x in loc_plt.get_yticks()/1000000)
loc_plt.set_xticklabels(loc_plt.get_xticklabels(),rotation = 45)



As hypothezized houses in a good location are much more expensive than the rest of the houses.


### Answers for my client:

We assume his houses are in the seven most expensive areas identified above.

Best time of the year to sell:

In [ ]:
# scatterplot of sell date and price of all houses
plt.style.use('fast')

sctr_plt5 = sns.scatterplot(x=df['sell_date'],y=df['price'])
sns.despine(right=True,top=True)

# formatting the plot
sctr_plt5.set_yticklabels('{:,.2f}'.format(x) + 'M' for x in sctr_plt5.get_yticks()/1000000)
sctr_plt5.set_title('Price and sell date for all houses'.title())
sctr_plt5.set_xlabel('Sell date')
plt.show()

Now let's see if this changes when we only look at the houses in his area of interest:

In [ ]:
# new dataframe with all houses in his area
expensive_areas = median_prices.head(7).index.to_list()
df_exp_areas = df[df['zipcode'].isin(expensive_areas)]
df_exp_areas.shape

In [ ]:
# scatterplot of sell date and price for houses in clients area
sctr_plt6 = sns.scatterplot(x=df_exp_areas['sell_date'],y=df_exp_areas['price'])
sns.despine(right=True,top=True)


sctr_plt6.set_yticklabels('{:,.2f}'.format(x) + 'M' for x in sctr_plt6.get_yticks()/1000000)
sctr_plt6.set_title('Price and sell date for houses in \n Clients area of interest'.title())
sctr_plt6.set_xlabel('Sell date')
plt.show()

print('Correlation between sell date and price:')
df_exp_areas[['sell_date','price']].corr().round(2)

We can see that the selling date does not have an big influence on the price.

**Recommendation:** There is no need to wait for a certain time of year to sell houses.

Next we will check if renovating a property is increasing its price:

In [ ]:
median_prices_reno = df.query('yr_renovated > 0').groupby('zipcode')['price'].median().sort_values(ascending=False)
median_prices_reno.rename('price_renovated',inplace=True)
median_prices_reno.head(8)

In [ ]:
#creating the df for the plot
df_median_prices = pd.merge(median_prices, median_prices_reno, left_index=True, right_index=True, how='left')
df_median_prices.eval('price_delta = price_renovated - price',inplace=True)
df_median_prices.reset_index(inplace=True)
df_median_prices_exp = df_median_prices[df_median_prices['zipcode'].isin(expensive_areas)]
# changing the color palette
pal = ['red' if value < 0 else 'blue' for value in df_median_prices_exp.sort_values('zipcode')['price_delta']]

# creating barplot with seaborn
br_plt = sns.barplot(data=df_median_prices_exp,x='zipcode',y='price_delta',palette=pal)
sns.despine(right=True,top=True)

br_plt.set_ylabel('Price difference')
br_plt.set_title('Price difference between renovated and unrenovated houses'.title())
plt.show()

In [ ]:
# printing the mean price difference 
df_median_prices_exp['price_delta'].median()

The median price difference between renovated and unrenovated houses is 125 000 $ which equates to 6 to 14% of the median house price.

**Recommendations:** A renovation is a good way of increasing the value of a house, of course only if the renovation costs are not to high.
Also buying houses that are undervalued and then renovating and selling them might be a good option. 

